# imports 

In [13]:
#imports
import lingpy
from levenshtein_dist_calc import LevenshteinDistanceCalculator
import nltk
from read_tab_files import TabFileReader
import bcubed
from collections import defaultdict

# Clustering

## testing

In [14]:
barb_cognacy = TabFileReader.tab_reader(
    "chl2024_barbacoandata/chl2023_barbacoan_cognacy.tab"
)
barb_forms = TabFileReader.tab_reader(
    "chl2024_barbacoandata/chl2023_barbacoan_forms.tab"
)

In [15]:
barb_word_list = TabFileReader.get_word_list(barb_forms)
barb_alphabet = TabFileReader.get_alphabet(barb_word_list)

In [16]:
taxa = list(barb_forms.iloc[0][1:])
taxa

['Awa Pit', "Cha'palaa", 'Guambiano', 'Totoro', 'Tsafiki']

In [30]:
cognates = list(barb_forms.iloc[1][1:])
cognates

['su', 'tu', '', '', 'to']

In [18]:
#calculate distance matrix from cognate list (integrate with levenshtein distance calc maybe)
def get_distancematrix(cognate_list):
    if 
    distance_matrix = [[0 for i in range(len(cognate_list))] 
                           for j in range(len(cognate_list))]
    for i, source in enumerate(cognate_list):
        for j, target in enumerate(cognate_list):
            if (source == "" 
                or target == ""):  # if one cognate in pair is missing, null value
                distance_matrix[i][j] = float("nan")
            else:
                distance_matrix[i][j] = nltk.edit_distance(source, target)/max(len(source),len(target))
    
    return distance_matrix

In [39]:
distance_matrix = get_distancematrix(cognates)

In [40]:
distance_matrix

[[0.0, 0.5, nan, nan, 1.0],
 [0.5, 0.0, nan, nan, 0.5],
 [nan, nan, nan, nan, nan],
 [nan, nan, nan, nan, nan],
 [1.0, 0.5, nan, nan, 0.0]]

In [32]:
cluster_dict_p2 = lingpy.flat_upgma(.2,distance_matrix,taxa)
cluster_dict_p5 = lingpy.flat_upgma(.5,distance_matrix,taxa)
print(cluster_dict_p2)
print(cluster_dict_p5)

{0: ['Awa Pit'], 1: ["Cha'palaa"], 2: ['Guambiano'], 3: ['Totoro'], 4: ['Tsafiki']}
{0: ['Awa Pit', "Cha'palaa"], 2: ['Guambiano'], 3: ['Totoro'], 4: ['Tsafiki']}


In [33]:
taxa

['Awa Pit', "Cha'palaa", 'Guambiano', 'Totoro', 'Tsafiki']

In [34]:
gold_clusters = list(barb_cognacy.iloc[1][1:])
gold_clusters

['45', '45', '', '', '45']

In [35]:
#gold file clustering to taxa:cluster dictionary transformer for bcubed comparisons
def gold_clust2taxa_dict(taxa, clusters):
    cluster_dict = {}
    for lang, cluster in zip(taxa, clusters):
        cluster_dict[lang] = set([cluster])
    return cluster_dict

gold_taxadict = gold_clust2taxa_dict(taxa, gold_clusters)

In [38]:
gold_taxadict

{'Awa Pit': {'45'},
 "Cha'palaa": {'45'},
 'Guambiano': {''},
 'Totoro': {''},
 'Tsafiki': {'45'}}

In [36]:
#lingpy.upgma_flat output to taxa:cluster dictionary transformer for bcubed comparisons
def upgma2taxa_dict(calculated_dict):
    output_clusters = {}
    for clus, langs in calculated_dict.items():
        for lang in langs:
            output_clusters[lang] = set(str(clus))
    return output_clusters

p2_taxa_dict = upgma2taxa_dict(cluster_dict_p2)
p5_taxa_dict = upgma2taxa_dict(cluster_dict_p5)

In [41]:
precision = bcubed.precision(p5_taxa_dict, gold_taxadict)
recall = bcubed.recall(p5_taxa_dict, gold_taxadict)
fscore = bcubed.fscore(precision, recall)
print(precision, recall)

1.0 0.5333333333333333


In [29]:
barb_cognacy.iloc[1]

0    1-21. the land
1                45
2                45
3                  
4                  
5                45
Name: 1, dtype: object

In [25]:
def prepare_comparison(threshold:float, gold_row:list, cognates:list, taxa:list)->tuple:
    distance_matrix = get_distancematrix(cognates)
    cluster_dict = lingpy.flat_upgma(threshold,distance_matrix,taxa)
    taxa_dict = upgma2taxa_dict(cluster_dict)
    gold_taxadict = gold_clust2taxa_dict(taxa, gold_row)
    return taxa_dict, gold_taxadict


In [56]:
def main(language_fam, fam_gold, threshold=4, remove_gaps = False):
    precision = 0
    recall = 0
    fscore = 0
    taxa = list(language_fam.iloc[0][1:])
    for i in range(1, len(language_fam)):
        cognates = list(language_fam.iloc[i][1:])
        gold_row = list(fam_gold.iloc[i][1:])
        if remove_gaps == True:
            if "" in gold_row:
                gold_row.remove("")
            if "" in cognates:
                cognates.remove("")
        taxa_dict, gold_taxadict = prepare_comparison(threshold, gold_row, cognates, taxa)
        precision += bcubed.precision(taxa_dict, gold_taxadict)
        recall += bcubed.recall(taxa_dict, gold_taxadict)
        fscore += bcubed.fscore(precision, recall)
    n_rows = len(language_fam)-1
    return precision/n_rows, recall/n_rows, fscore/n_rows
    

In [57]:
print(main(barb_forms, barb_cognacy))
print(main(barb_forms, barb_cognacy, remove_gaps=True))

(0.9355848724714162, 0.5966578715919097, 138.9135688055214)
(0.8454353562005275, 0.7675901495162708, 153.83834390780655)
